In [1]:
%pip install -r requirements.txt

import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [148]:
#interest rate

df = pd.read_csv('./data/before-convert/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

df = df.rename(columns = {'LOCATION':'location'})
df = df.drop('Country', axis=1)

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
longTerm = longTerm.rename(columns = {'Interest-Rate':'interest_rate_long_term'})
longTerm = longTerm.drop('Subject', axis=1)

shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
shortTerm = shortTerm.rename(columns = {'Interest-Rate':'interest_rate_short_term'})
shortTerm = shortTerm.drop('Subject', axis=1)


immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')
immediateTerm = immediateTerm.rename(columns = {'Interest-Rate':'interest_rate_immediate_term'})
immediateTerm = immediateTerm.drop('Subject', axis=1)

df = pd.merge(longTerm, shortTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = pd.merge(df, immediateTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = df[[c for c in df.columns if not c.endswith('_delme')]]

df.to_csv('./data/interest_rate.csv', index=False)

In [229]:
#cpi

#au
df_au = pd.read_csv('./data/before-convert/cpi_au.csv')

df_au['Date'] = pd.DatetimeIndex(df_au['TIME'])
df_au = df_au.drop('TIME', axis=1)

df_au['Date'] = pd.to_datetime(df_au['Date']).dt.to_period('M')
df_au = df_au.set_index('Date').resample('M').interpolate()
df_au['location'] = 'AUS'
df_au['year'] = pd.DatetimeIndex(df_au.index.to_timestamp()).year
df_au['month'] = pd.DatetimeIndex(df_au.index.to_timestamp()).month

df_au.reset_index(drop=True, inplace=True)


#us
df_us = pd.read_csv('./data/before-convert/cpi_us.csv')

df_final_us = pd.DataFrame(columns=['year','month'])

for index, row in df_us.iterrows():
    for i in range(12):
        df_final_us = pd.concat([df_final_us, pd.DataFrame.from_records([{'year':row[0],'month':i+1, 'cpi-index':row[i+1]}])])

df_final_us['location'] = 'USA'
df_final_us = df_final_us.dropna()
df_final_us['year'] = df_final_us['year'].astype(int)
    
df = pd.concat([df_au, df_final_us])

df.to_csv('./data/cpi.csv', index=False)

In [258]:
#energy_price

#au

df_au = pd.read_csv('./data/before-convert/energy_price_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).year
df_au['month'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).month

df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').median()
df_au.reset_index(drop=True, inplace=True)

df_au['location'] = 'AUS'
df_au = df_au.rename(columns = {'Value':'energy_price'})
df_au['year'] = df_au['year'].astype(int)
df_au['month'] = df_au['month'].astype(int)

#us

df_us = pd.read_csv('./data/before-convert/energy_price_us.csv')

df_us['year'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).month

df_us = df_us.drop('date', axis=1)
df_us['location'] = 'USA'
df_us = df_us.rename(columns = {'value':'energy_price'})

df_us.head(100)

df = pd.concat([df_au, df_us])

df.to_csv('./data/energy_price.csv', index=False)

In [257]:

df_au = pd.read_csv('./data/before-convert/energy_price_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).year
df_au['month'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).month

df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').median()
df_au.reset_index(drop=True, inplace=True)

df_au['location'] = 'AUS'
df_au = df_au.rename(columns = {'Value':'energy_price'})
df_au['year'] = df_au['year'].astype(int)
df_au['month'] = df_au['month'].astype(int)

df_au.head(1000)

,energy_price,year,month,location
0,30.430,2003,1,AUS
1,31.845,2003,2,AUS
2,29.800,2003,3,AUS
3,25.380,2003,4,AUS
4,26.170,2003,5,AUS
...,...,...,...,...
234,108.700,2022,7,AUS
235,101.290,2022,8,AUS
236,95.790,2022,9,AUS
237,93.320,2022,10,AUS
